# Load Data

# Run tuning

In [ ]:
import optuna
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_score

# Define the objective function for the Optuna study
def objective(trial):
    # Define the hyperparameter space
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'eta': trial.suggest_float('eta', 0.01, 0.32),
        'subsample': trial.suggest_categorical('subsample', [0.7, 0.8, 0.9]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7, 0.8, 0.9]),
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
    }
    
    # Initialize the model with the current set of hyperparameters
    model = XGBRegressor(**params)
    
    # Use LeaveOneOut as the cross-validation strategy
    loo = LeaveOneOut()
    
    # Calculate the cross-validated scores
    scores = cross_val_score(model, X_train, y_train, cv=loo, scoring='neg_mean_absolute_error')
    
    # The objective is to maximize the negative mean absolute error (as Optuna minimizes the objective function)
    return np.mean(scores)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # Adjust n_trials to your preference

# Print the best parameters and the best score
print(f"Best parameters found: {study.best_params}")
print(f"Best MAE score: {study.best_value}")